# Demo Data Model

![Demo Data Model](visuals/DemoDataModel.png)

# Populate Customer Data in Amazon Aurora PostgreSQL

In [ ]:
#pip install psycopg2

In [ ]:
import os
PG_HOST = os.environ.get("PG_HOST")
PG_SECRET_ARN = os.environ.get("PG_SECRET_ARN")
print(PG_SECRET_ARN)

In [ ]:
import boto3
import json

client = boto3.client('secretsmanager')
response = client.get_secret_value(SecretId=PG_SECRET_ARN)
database_secrets = json.loads(response['SecretString'])
pwd = database_secrets['password']

In [ ]:
import psycopg2

# Connect to database
conn = psycopg2.connect(
    user='postgres',
    dbname='postgres',
    password=pwd,
    host=PG_HOST, #'demo-cluster.cluster-{id}.{region}.rds.amazonaws.com'
    port='5432'
)

# Set autocommit for DDL statements
conn.autocommit = True
cur = conn.cursor()

sql_statements = []
with open('./sql/aurorapg.sql', 'r') as file:
    # Split on semicolons but ignore empty statements
    sql_statements = [stmt.strip() for stmt in file.read().split(';') if stmt.strip()]

# Execute each statement
for sql in sql_statements:
    print(f"Executing SQL: {sql}")
    cur.execute(sql)

# Clean up
cur.close()
conn.close()

# Generate Financial Data (invoices) in Amazon DynamoDB 

In [ ]:
import boto3
import secrets

# Initialize DynamoDB client
dynamodb = boto3.resource('dynamodb')

table_name = 'invoices'
table = dynamodb.Table(table_name)

# Items to add
items = []
for i in range(1, 101):
    # Generate random within a range (10248, 11077)
    range_size = 11077 - 10248 + 1
    order_id = 10248 + secrets.randbelow(range_size)

    # Generate random invoice number (1000, 9999)
    random_num = secrets.randbelow(1000000)  # Adjust the range as needed
    invoice_number = f"INV_{random_num}"

    # Generate random total between 100 and 10000
    range_size_total = 10000 - 100 + 1
    total = 100 + secrets.randbelow(range_size_total)

    # For status
    statuses = ['DRAFT', 'SUBMITTED', 'AUTHORISED', 'PAID', 'VOIDED']
    status = statuses[secrets.randbelow(len(statuses))]

    # Create and append the item
    items.append({
        'order_id': order_id,
        'invoice_number': invoice_number,
        'status': status,
        'total': total
    })

# Use batch_writer for better performance
with table.batch_writer() as batch:
    for item in items:
        batch.put_item(Item=item)
        print(f'Added item: {item}')